In [2]:
!nvidia-smi

Tue Nov  3 12:11:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   35C    P2    67W / 280W |  11688MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 41%   34C    P8    22W / 280W |  16954MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [4]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data

import torch.optim as optim

sys.path.append('/home/sreena/abhishek/AliProducts/architectures/')
sys.path.append('/home/sreena/abhishek/AliProducts/Helper/')
from dataloader import mydataset, create_prime_dict 
from trainer import train, test_classify, eval_classify
from resnet_models import ResNet,Bottleneck
from Load_model import load
from plot_curves import plot_loss, plot_acc

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


**Dataloading Scheme**

In [6]:
trainlist = '/home/sreena/abhishek/AliProducts/lists/balanced_training_list.txt'
validlist = '/home/sreena/abhishek/AliProducts/lists/balanced_validation_list.txt'

In [7]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  43043


In [8]:
#### Train Dataloader #### 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 256, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 128, num_workers=16,pin_memory=True)

**Model Definition**

In [9]:
model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 43043)

model = nn.DataParallel(model,device_ids=[0,1,2,3]).to(device)
model

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

**Hyperparameters**

In [10]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 30, gamma = 0.1)

# Epochs
num_Epochs = 120

In [11]:
modelpath = '../saved_model_checkpoints/AliProducts/Apro_AA_Cutout_R50_copy'

In [13]:
train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, device, epochs = num_Epochs)

loss 11.596273498535156
loss 22.748490953445433
loss 33.7499858379364
loss 44.63201066017151
loss 55.399776000976566
loss 66.09168650627136
loss 76.71288962364197
loss 87.26547222137451
loss 97.75624344825745
loss 108.18809692382813
loss 118.54619094848633
loss 128.87068860054015
loss 139.1414221763611
loss 149.36863698005675
loss 159.5479545688629
loss 169.70107124328612
loss 179.8088912677765
loss 189.88139450073243
loss 199.91470337867736
loss 209.90465364456176
loss 219.83775647163392
loss 229.75090594291686
loss 239.622932510376
loss 249.44084263801574
loss 259.21742690086364
loss 268.9465227794647
loss 278.62980268478395
loss 288.2564816665649
loss 297.8421774673462
loss 307.37411750793456
loss 316.83872997283936
loss 326.2721576786041
loss 335.63843397140505
loss 344.90771324157714
loss 354.11675723075865
loss 363.259090719223
loss 372.31930208206177
Epoch:  1
training loss =  10.060732050922189
loss 8.81529706954956
loss 17.559094381332397
loss 26.270381355285643
loss 34.891597

loss 10.240632075071336
loss 12.069840326309205
loss 13.938920387029647
loss 15.809349184036256
loss 17.729643260240554
loss 19.65984716773033
loss 21.615733636617662
loss 23.604098544120788
loss 25.571662653684616
loss 27.5611525785923
loss 29.612983305454254
loss 31.682136840820313
loss 33.762827520370486
loss 35.87427238225937
loss 37.97296233534813
loss 40.10644050359726
loss 42.27728068351745
loss 44.46165667295456
loss 46.65139654159546
loss 48.924599041938784
loss 51.16887447595596
loss 53.41809599876404
loss 55.67744483232498
loss 57.96171327114105
loss 60.26143671631813
loss 62.60909354448319
loss 64.93766951560974
loss 67.26440973162651
loss 69.60172724366188
loss 71.94569890141487
loss 74.26609712719917
loss 76.603294993639
Epoch:  10
training loss =  2.07091903921276
loss 1.6091773283481599
loss 3.25008682012558
loss 4.900350543260575
loss 6.536472481489182
loss 8.215294930934906
loss 9.943226191997528
loss 11.699727711677552
loss 13.495151205062866
loss 15.316183601617814


loss 7.115673738718033
loss 8.625575969219208
loss 10.139751901626587
loss 11.693225774765015
loss 13.271277552843094
loss 14.892294850349426
loss 16.528485676050185
loss 18.18512448787689
loss 19.86912027001381
loss 21.587797782421113
loss 23.30888095974922
loss 25.093033586740493
loss 26.859638257026674
loss 28.6693274474144
loss 30.526070829629898
loss 32.40160165309906
loss 34.296898450851444
loss 36.1625917840004
loss 38.11834899663925
loss 40.03809939026833
loss 41.975172213315965
loss 43.952169661521914
loss 45.949269852638245
loss 47.9191490828991
loss 49.9702697122097
loss 52.01086965560913
loss 54.09569990277291
loss 60.32009246706963
loss 62.43519679546356
loss 64.5455368220806
loss 66.63687239527702
Epoch:  19
training loss =  1.8015875756853124
loss 1.377556298971176
loss 2.747803041934967
loss 4.18627625465393
loss 5.616000086665154
loss 7.073282825350762
loss 8.538827316164971
loss 10.06718615591526
loss 11.63506234228611
loss 13.223551059365272
loss 14.801507814526557
l

loss 21.821868438720703
loss 23.493823701143263
loss 25.166573407649995
loss 26.83883920788765
loss 28.546021124124525
loss 30.302893849611284
loss 32.090433623790744
loss 33.87741080880165
loss 35.68845121026039
loss 37.541359672546385
loss 39.41930737257004
loss 41.292046941518784
loss 43.18518195033074
loss 45.0755021250248
loss 46.96958045840263
loss 48.930478641986845
loss 50.87628025650978
loss 52.87362020134926
loss 54.881718602180484
loss 56.90013919472695
loss 58.93394629716873
loss 60.98397824764252
loss 63.024182909727095
Epoch:  28
training loss =  1.7040564773344498
loss 1.2949912351369859
loss 2.6094205850362777
loss 3.904471293091774
loss 5.248545911312103
loss 6.615122691392899
loss 8.031847025752068
loss 9.421749215126038
loss 10.873244837522506
loss 12.333633185625077
loss 13.849200299978257
loss 15.380253802537919
loss 16.92630555033684
loss 18.512742327451704
loss 20.11649008154869
loss 21.77985515832901
loss 23.43275866866112
loss 25.15217633008957
loss 26.85566812

loss 3.592467807382345
loss 3.9525555197894575
loss 4.306700756400824
loss 4.672283150553703
loss 5.0455682802200315
loss 5.414100653380156
loss 5.780653134286403
loss 6.14909411072731
loss 6.521903623342514
loss 6.911194252967834
loss 7.28446801006794
loss 7.649304782003164
loss 8.030895725637674
loss 8.397678276002408
loss 8.783211009055377
loss 9.159937547445297
loss 9.528753596693278
loss 9.911121421009302
loss 10.286036720126868
loss 10.669335954487323
loss 11.052096256613732
loss 11.437338532209397
loss 11.830316160619258
loss 12.222550420165062
loss 12.597692072093487
loss 12.994150563180447
loss 13.374798817038537
loss 13.760267570018769
Epoch:  37
training loss =  0.3720556120683409
loss 0.3438410234451294
loss 0.6666810709238052
loss 0.9955715049803257
loss 1.3345137426257134
loss 1.6599659004807472
loss 1.99666972219944
loss 2.328630320727825
loss 2.6755869929492473
loss 3.014435938894749
loss 3.358457781970501
loss 3.7029701612889765
loss 4.040652023404837
loss 4.3905094993

...

loss 0.9107028257846832
loss 1.0628441473096608
loss 1.215991049259901
loss 1.3699820681661368
loss 1.5190169547498227
loss 1.6664428175985813
loss 1.8187453116476535
loss 1.969334817454219
loss 2.1221108543872833
loss 2.277561332657933
loss 2.4302565172314643
loss 2.5811625749617817
loss 2.7368347331136467
loss 2.8915878038108347
loss 3.0417533600330353
loss 3.196990552023053
loss 3.3514173167943953
loss 3.5074325907230377
loss 3.6645260103791952
loss 3.8187488407641648
loss 3.973302040100098
loss 4.125540791898966
loss 4.276174477860332
loss 4.4320342655479905
loss 4.592623677775264
loss 4.749778665676713
loss 4.90563898421824
loss 5.064485765397548
loss 5.219683122783899
loss 5.375457850843668
loss 5.530375098064542
loss 5.689581538811326
Epoch:  82
training loss =  0.1538188605596185
loss 0.15011242538690567
loss 0.3031700186431408
loss 0.45302725188434123
loss 0.6063497342169285
loss 0.7582357870787382
loss 0.9106510401517153
loss 1.058571508154273
loss 1.211220958456397
loss 1.36

loss 0.5948178373277188
loss 0.7406240758299828
loss 0.8877898471057415
loss 1.0351487765461207
loss 1.1852265764772891
loss 1.3341235259920359
loss 1.4831130089610816
loss 1.6309335757791996
loss 1.7803427877277136
loss 1.9297559820115566
loss 2.07469307564199
loss 2.222675762027502
loss 2.3735160394757986
loss 2.521510680243373
loss 2.6692293187975884
loss 2.8189353804290294
loss 2.9681291246414183
loss 3.1182262459397316
loss 3.2641426254063846
loss 3.4087606217712163
loss 3.5586730621010063
loss 3.709466679021716
loss 3.860292393565178
loss 4.005343318358063
loss 4.151535642817617
loss 4.29549785874784
loss 4.443560472726822
loss 4.5907552190124985
loss 4.740141266733408
loss 4.891629275083542
loss 5.041501141041517
loss 5.189092596843839
loss 5.335192663669586
loss 5.48280342809856
Epoch:  91
training loss =  0.1481661971569029
loss 0.14786832965910435
loss 0.2959196385741234
loss 0.4432438925653696
loss 0.5878531485795975
loss 0.7333047211170196
loss 0.8832049237191677
loss 1.031

loss 2.320234564319253
loss 2.4637443847209215
loss 2.6113341065496205
loss 2.7582895012944935
loss 2.9014957401156427
loss 3.050470738038421
loss 3.197352222278714
loss 3.340227492898703
loss 3.4906456254422666
loss 3.6370447777956723
loss 3.7815701991319655
loss 3.926764774620533
loss 4.06954260237515
loss 4.215070466846227
loss 4.359187062010169
loss 4.502245241478086
loss 4.645917734578251
loss 4.79191465139389
loss 4.934772073924542
loss 5.082073455601931
loss 5.2288584762066606
loss 5.374006948247552
Epoch:  100
training loss =  0.1452473922699714
loss 0.1464264427870512
loss 0.2924574124068022
loss 0.4374022881686688
loss 0.5855838909745217
loss 0.7322919421643018
loss 0.8793919838964939
loss 1.0232534839957952
loss 1.1644035306572915
loss 1.3098620063811541
loss 1.4569789159297943
loss 1.6021603168547154
loss 1.7471358727663755
loss 1.895096865221858
loss 2.0417094357311725
loss 2.1896604904532433
loss 2.3339073872566223
loss 2.476994948834181
loss 2.62079313993454
loss 2.76576

loss 2.1957432712614535
loss 2.336894762814045
loss 2.48303425706923
loss 2.6294858043640854
loss 2.777762033045292
loss 2.9242395881563423
loss 3.0691613382846117
loss 3.2132343860715626
loss 3.356097894087434
loss 3.499559233263135
loss 3.641158530935645
loss 3.7852479539066555
loss 3.9290644263476135
loss 4.076686114147305
loss 4.223961908146739
loss 4.66461874268949
loss 4.811051576733589
loss 4.95643229611218
loss 5.098663999512792
loss 5.244237293824553
loss 5.389251304864883
Epoch:  109
training loss =  0.14564384175763478
loss 0.14552015393972398
loss 0.2878163132816553
loss 0.42822190530598164
loss 0.5735831865668297
loss 0.7190479693561792
loss 0.8637972718477249
loss 1.0084073204547166
loss 1.1537326642125845
loss 1.3005656941980124
loss 1.448780653104186
loss 1.596806765049696
loss 1.741872677281499
loss 1.8885312653332949
loss 2.033632073029876
loss 2.181970821991563
loss 2.3239343336969616
loss 2.468638234585524
loss 2.609807977229357
loss 2.760267184376717
loss 2.9060191

loss 2.601665861532092
loss 2.748570570498705
loss 2.89619537897408
loss 3.0404009366035463
loss 3.1874407927691935
loss 3.3303110478818416
loss 3.4743074468523263
loss 3.618799766972661
loss 3.765964159891009
loss 3.9118186365813017
loss 4.054118996933102
loss 4.198862951770425
loss 4.345696912109852
loss 4.486634963676334
loss 4.629607694745064
loss 4.769662273004651
loss 4.914984390586614
loss 5.060490246787667
loss 5.2041925232112405
loss 5.350745811089873
Epoch:  118
training loss =  0.14464505814520343
loss 0.14609684586524962
loss 0.2919485978037119
loss 0.43982272371649744
loss 0.5828959207981825
loss 0.7262066801637411
loss 0.8739999016374349
loss 1.016682099327445
loss 1.1590346924960613
loss 1.3002948114275932
loss 1.4447877063602208
loss 1.5920300460606813
loss 1.73716973900795
loss 1.8801751619577407
loss 2.0237797944247724
loss 2.1725644730776548
loss 2.31429494895041
loss 2.4586272870749237
loss 2.6008247376233338
loss 2.745375006720424
loss 2.8869598414748907
loss 3.035

In [14]:
#### Load saved model from checkpoint  #####
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


KeyError: 'Validation_Loss_List'

In [15]:
#### Plot Loss Curves ####/
plot_loss(epoch, train_loss, v_loss, title = 'Autoaugment_Cutout_Resnet50')
plot_acc(epoch, v_acc)

NameError: name 'epoch' is not defined

In [20]:
t_loss,top1_t_acc, top5_t_acc =test_classify(model, train_dataloader, criterion, device)
print('Training Loss: {:.4f}\tTop 1 Training Accuracy: {:.4f}\t Top 5 Training Accuracy: {:.4f}'.format(t_loss, top1_t_acc, top5_t_acc))

Training Loss: 0.4860	Top 1 Training Accuracy: 0.8849	 Top 5 Training Accuracy: 0.9747


In [16]:
v_loss, top1_acc, accuracy_dict= eval_classify(model, validation_dataloader, criterion, device)
print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}\nAccuracy:{}\t'.format(v_loss, top1_acc, accuracy_dict))

Validation Loss: 1.4769	Top 1 Validation Accuracy: 0.7929
Accuracy:defaultdict(<class 'int'>, {'Top 1 Accuracy': 79.29395913272513, 'Top 5 Accuracy': 89.31768459332126, 'Top 10 Accuracy': 89.80188226493657, 'Top 20 Accuracy': 90.15404650911279, 'Top 30 Accuracy': 90.33677536517607, 'Top 50 Accuracy': 90.56591870255752, 'Top 100 Accuracy': 90.95548490740761})	


In [24]:
len(validation_dataloader.dataset)

443827

In [25]:
len(train_dataloader.dataset)

948822